In [59]:
import pandas as pd 
import ast
import re
import numpy as np
import nltk
import json
from datetime import datetime
from datetime import timedelta
from textblob import TextBlob
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.util import ngrams
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy
import pyLDAvis.gensim_models
# import en_core_web_md
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ivankoh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ivankoh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ivankoh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/ivankoh/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [60]:
df = pd.read_csv("/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4222/Data/kickstarter_train.csv")

In [61]:
# convert 'video' to a binary categorical variable
df['video'].value_counts()
df['has_video'] = df['video'].apply(lambda x: 0 if pd.isnull(x) else 1)
df['has_video'].value_counts()

# df['faq_count'] = df['faq_count'].apply(lambda x: float(x.replace(",", ""))) # transfer over to feature eng

1    17881
0     6875
Name: has_video, dtype: int64

In [62]:
# extract main category and subcategory from category variable
# df['subcategory'] = df['category'].apply(lambda x: json.loads(x)['slug'].split("/")[1] if len(json.loads(x)['slug'].split("/")) > 1 else None)
#df['category'] = df['category'].apply(lambda x: json.loads(x)['slug'].split("/")[0])
# df['location'] = df['location'].apply(lambda x: json.loads(x)['country'])

## NLP features

In [63]:
# Text cleaning for: rewards, description, description story, description risks

def clean_text(df):
    def process_rewards(corpus):
    
        corpus_processed = []
        for row in corpus:
            row_processed = ""
            row = row.replace("\\n", " ")
            row = ast.literal_eval(row)

            for dict in row:
                row_processed += dict['rewards'].lower() + ' '

            
            row_processed = row_processed.replace("//",'')
            row_processed = re.sub(r'[^\w\s]', '', row_processed) # remove punctuation
            corpus_processed.append(row_processed)

        return corpus_processed
    
    def process_description_story(corpus):
        corpus_processed = []
        for row in corpus:
            row = str(row)
            row_processed = row.replace("\r", " " )
            row_processed = row_processed.replace("\n", " " )
            row_processed = re.sub(r'[^\w\s]', '', row_processed) # remove punctuation
            corpus_processed.append(row_processed if not pd.isnull(row_processed) else "") # handle NA

        return corpus_processed

    df["rewards_processed"] = process_rewards(df["rewards"])
    df["description_processed"] = process_description_story(df["description"])
    df["description_story_processed"] = process_description_story(df["description_story"])
    df["description_risks_processed"] = process_description_story(df["description_risks"])

    return df


In [64]:
class LemmatizeTokenizer(object):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
    def __call__(self, text):
        return [self.lemmatizer.lemmatize(word) for word in word_tokenize(text)]

In [65]:
# need to return the vectorizer to use on the test data to transform it 
def generate_nlp_features(df):
    
    # Rewards

    vect_rewards = TfidfVectorizer( 
        tokenizer=LemmatizeTokenizer(),
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=100, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )

    rewards_processed = pd.Series(df["rewards_processed"])
    tfidf_fit_rewards = vect_rewards.fit(rewards_processed)
    rewards_tfidf_array = tfidf_fit_rewards.transform(rewards_processed).toarray()
    rewards_tfidf_df = pd.DataFrame(rewards_tfidf_array)
    rewards_tfidf_df.columns = list(map(lambda x : "rewards_" + str(x), rewards_tfidf_df.columns))
    df = pd.merge(df, rewards_tfidf_df , left_index=True, right_index=True)
    

    # Description

    vect_description = TfidfVectorizer( 
        tokenizer=LemmatizeTokenizer(),
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=100, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )

    description = pd.Series(df["description_processed"])
    tfidf_fit_description = vect_description.fit(description)
    description_tfidf_array = tfidf_fit_description.transform(description).toarray()
    description_tfidf_df = pd.DataFrame(description_tfidf_array)
    description_tfidf_df.columns = list(map(lambda x : "description_" + str(x), description_tfidf_df.columns))
    df = pd.merge(df, description_tfidf_df , left_index=True, right_index=True)


    # Description Story

    vect_description_story = TfidfVectorizer( 
        tokenizer=LemmatizeTokenizer(),
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=100, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )
    
    description_story_processed = pd.Series(df["description_story_processed"])
    tfidf_fit_description_story_processed = vect_description_story.fit(description_story_processed)
    description_story_processed_tfidf_array = tfidf_fit_description_story_processed.transform(description_story_processed).toarray()
    description_story_tfidf_df = pd.DataFrame(description_story_processed_tfidf_array)
    description_story_tfidf_df.columns = list(map(lambda x : "description_story_" + str(x), description_story_tfidf_df.columns))
    df = pd.merge(df, description_story_tfidf_df , left_index=True, right_index=True)


    # Description Risks

    vect_description_risks = TfidfVectorizer( 
        tokenizer=LemmatizeTokenizer(),
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=100, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )

    description_risks_processed = pd.Series(df["description_risks_processed"])
    tfidf_fit_description_risks_processed = vect_description_risks.fit(description_risks_processed)
    description_risks_processed_tfidf_array = tfidf_fit_description_risks_processed.transform(description_risks_processed).toarray()
    description_risks_tfidf_df = pd.DataFrame(description_risks_processed_tfidf_array)
    description_risks_tfidf_df.columns = list(map(lambda x : "description_risks_" + str(x), description_risks_tfidf_df.columns))
    df = pd.merge(df, description_risks_tfidf_df , left_index=True, right_index=True)

    return (df, tfidf_fit_rewards, tfidf_fit_description, tfidf_fit_description_story_processed, tfidf_fit_description_risks_processed)


def nlp_transform_test(df, tfidf_fit_rewards, tfidf_fit_description, tfidf_fit_description_story_processed, tfidf_fit_description_risks_processed):
    rewards_tfidf_array = tfidf_fit_rewards.transform(df['rewards_processed']).toarray()
    rewards_tfidf_df = pd.DataFrame(rewards_tfidf_array)
    rewards_tfidf_df.columns = list(map(lambda x : "rewards_" + str(x), rewards_tfidf_df.columns))
    df = pd.merge(df, rewards_tfidf_df , left_index=True, right_index=True)
    
    description_tfidf_array = tfidf_fit_description.transform(df['description_processed']).toarray()
    description_tfidf_df = pd.DataFrame(description_tfidf_array)
    description_tfidf_df.columns = list(map(lambda x : "description_" + str(x), description_tfidf_df.columns))
    df = pd.merge(df, description_tfidf_df , left_index=True, right_index=True)
    
    description_story_processed_tfidf_array = tfidf_fit_description_story_processed.transform(df['description_story_processed']).toarray()
    description_story_tfidf_df = pd.DataFrame(description_story_processed_tfidf_array)
    description_story_tfidf_df.columns = list(map(lambda x : "description_story_" + str(x), description_story_tfidf_df.columns))
    df = pd.merge(df, description_story_tfidf_df , left_index=True, right_index=True)
    
    description_risks_processed_tfidf_array = tfidf_fit_description_risks_processed.transform(df['description_risks_processed']).toarray()
    description_risks_tfidf_df = pd.DataFrame(description_risks_processed_tfidf_array)
    description_risks_tfidf_df.columns = list(map(lambda x : "description_risks_" + str(x), description_risks_tfidf_df.columns))
    df = pd.merge(df, description_risks_tfidf_df , left_index=True, right_index=True)
    
    return df


In [66]:
for x in df.columns:
    print(x)

id
name
description
description_story
description_risks
rewards
category
goal
deadline
location
state
staff_pick
video
launched_at
created_at
rewards_processed
description_processed
description_story_processed
description_risks_processed
has_video


## Rewards Features

In [67]:
# Remove rewards that got cut off while scraping and closes the dictionary
def clean_rewards(reward_list):
    try:
        tiers = ast.literal_eval(reward_list)
    except SyntaxError as e:
        i = len(reward_list) -1
        while reward_list[i] != "{":
            i -= 1
        reward_list = reward_list[:(i-1)]
        reward_list += ']'
    return reward_list
    
def create_rewards_tiers(df):
    # df["rewards"] = df["rewards"].apply(lambda x : clean_rewards(x))
    df["reward_tiers"] = df["rewards"].apply(lambda x : len(ast.literal_eval(x)))
    df = move_reward_tiers(df)
    return df

def create_min_max_reward(df):
    df["min_reward"] = 0
    df["max_reward"] = 0

    for i in range(len(df)):
        print(i)
        all_reward_amount = []
        reward = df.iloc[i, 5]
        dict_list = ast.literal_eval(reward)
        for dictionary in dict_list:
            values_string = str(dictionary.values())
            reward_title_SGD = re.search(r"(S)?\$\s?\d{1,3}(,\d{1,3})?", values_string) #S$
            if reward_title_SGD:
                reward_amount = re.search(r"\d{1,3}(,\d{1,3})?", reward_title_SGD.group())
            else:
                 print("Error: No reward in SGD found!")   

            if reward_amount is not None:
                integer_amount = re.sub("[^\d\.]", "", reward_amount.group()) # remove comma in large amounts of money
                all_reward_amount.append(int(integer_amount))
            else:
                pledged_without_reward = re.search(r"Pledge without a reward", values_string) # check if have "Pledge without a reward"
                if pledged_without_reward:
                    all_reward_amount.append(0)
        if len(all_reward_amount) > 0:
            df["min_reward"][i] = min(all_reward_amount)
            df["max_reward"][i] = max(all_reward_amount)
    print("create min max reward done, now moving columns")
    df = move_min_max_reward(df)
    return df

# Rearange reward_tiers column to the right of rewards
def move_reward_tiers(df):
    cols = df.columns.tolist()
    cols = cols[:6] + [cols[-1]] + cols[6:-1]
    df = df[cols]
    return df

# Rearange min and max reward columns to the right of reward_tiers
def move_min_max_reward(df):
    cols = df.columns.tolist()
    cols = cols[:7] + cols[-2:] + cols[7:-2]
    df = df[cols]
    return df

<>:37: DeprecationWarning: invalid escape sequence \d
<>:37: DeprecationWarning: invalid escape sequence \d
/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_66965/3232248864.py:37: DeprecationWarning: invalid escape sequence \d
  integer_amount = re.sub("[^\d\.]", "", reward_amount.group()) # remove comma in large amounts of money


## Sentiment Features

In [68]:
'''
Polarity is float which lies in the range of [-1,1] where 1 means positive statement and -1 means a negative statement. Subjective sentences generally refer to personal opinion, emotion or judgment whereas objective refers to factual information. Subjectivity is also a float which lies in the range of [0,1].
'''
def generate_sentiment_features(df):
    df = df.dropna(subset=['description_story_processed', 'description_risks_processed', 'description_processed', 'rewards_processed']) # NOTE: put at top with other dropnas from other features?
    df["description_story_polarity"] = df["description_story_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.polarity)
    df["description_story_subjectivity"] = df["description_story_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.subjectivity)
    df["description_polarity"] = df["description_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.polarity)
    df["description_subjectivity"] = df["description_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.subjectivity)
    df["description_risks_polarity"] = df["description_risks_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.polarity)
    df["description_risks_subjectivity"] = df["description_risks_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.subjectivity)
    df["rewards_polarity"] = df["rewards_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.polarity)
    df["rewards_subjectivity"] = df["rewards_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.subjectivity)          
    return df

In [69]:
def generate_word_count_features(df): # omitted description due to word limit, word count likely similar for all projects
    df['description_story_word_count'] = df["description_story_processed"].apply(lambda x: len(str(x).split(" ")))
    df['description_risks_word_count'] = df["description_risks_processed"].apply(lambda x: len(str(x).split(" ")))
    df['rewards_word_count'] = df["rewards_processed"].apply(lambda x: len(str(x).split(" ")))
    return df

## One-hot Encoding of Categorical Variables

when to drop one of the OHE columns: 
https://stats.stackexchange.com/questions/231285/dropping-one-of-the-columns-when-using-one-hot-encoding

In [70]:
def ohe_transform(enc, col, df):
    transformed = enc.transform(df[col].to_numpy().reshape(-1, 1))
    #Create a Pandas DataFrame of the hot encoded column
    ohe_df = pd.DataFrame(transformed, columns=enc.get_feature_names_out())
    #concat with original data
    data = pd.concat([df, ohe_df], axis=1)#.drop([col], axis=1)
    return data

def ohe_fit(col, df):
    enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
    enc.fit(df[col].unique().reshape(-1, 1))
    return (ohe_transform(enc, col, df), enc)

hold out on topic modelling first because it is unsupervised algorithm

<h2> Generate Duration Feature </h2>

In [71]:
def generate_duration_features(df):
    df['deadline'] = df['deadline'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df['launched_at'] = df['launched_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df['created_at'] = df['created_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df['funding_duration_in_days'] = (df['deadline'] - df['launched_at'].dt.tz_localize(None))
    df['funding_duration_in_days'] = df['funding_duration_in_days'].apply(lambda x: x / timedelta(days=1))
    df['preparation_duration_in_days'] = (df['launched_at'] - df['created_at'].dt.tz_localize(None))
    df['preparation_duration_in_days'] = df['preparation_duration_in_days'].apply(lambda x: x / timedelta(days=1))
    
    return df

## Combine all feature generating functions

In [72]:
result = create_rewards_tiers(df)
print("Rewards tiers done")
result = create_min_max_reward(result)
print("mins max rewards done")
#result = clean_text(result)
#print("clean text done")
result, tfidf_fit_rewards, tfidf_fit_description, tfidf_fit_description_story_processed, tfidf_fit_description_risks_processed = generate_nlp_features(result)
print("generate_nlp_features done")
result = generate_sentiment_features(result)
print("generate_sentiment_features done")
result = generate_word_count_features(result)
print("generate_word_count_features done")
result, category_encoder = ohe_fit('category', result) #use encoder to fit train data
print("ohe_fit category done")
# THIS IS WRONG NEED TO CLEAN LOCATION
result, location_encoder = ohe_fit('location', result) #use encoder to fit train data
print("ohe_fit location done")
result = generate_duration_features(result)
print("generate_duration_features done")

SyntaxError: EOL while scanning string literal (<unknown>, line 1)

## Apply same feature engineering on Test Data

In [ ]:
test = pd.read_csv("/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4222/Data/kickstarter_test.csv")
test = create_rewards_tiers(test)
test = create_min_max_reward(test)
#test = clean_text(test)
test['has_video'] = test['video'].apply(lambda x: 0 if pd.isnull(x) else 1)
# test['faq_count'] = test['faq_count'].apply(lambda x: float(x.replace(",", ""))) # transfer over to feature eng
#test = generate_nlp_features(test)
test = nlp_transform_test(test, tfidf_fit_rewards, tfidf_fit_description, tfidf_fit_description_story_processed, tfidf_fit_description_risks_processed)
test = generate_sentiment_features(test)
test = generate_word_count_features(test)
test = generate_duration_features(test)

#encoders
test = ohe_transform(category_encoder, 'category', test)
test = ohe_transform(location_encoder, 'location', test)

In [ ]:
print(len(test.columns))
print(len(result.columns))

for i in result.columns:
    if i not in test.columns:
        print(i)

## Export

In [ ]:
import time 

result.to_csv(f'data/train/kickstarter_train_final_{time.strftime("%Y%m%d-%H%M%S")}.csv', index=False)
test.to_csv(f'data/test/kickstarter_test_final_{time.strftime("%Y%m%d-%H%M%S")}.csv', index=False)

In [ ]:
#since there could be more than one categories for each project, create new features to split the categories.
# df['new_category'] = df.category.str.split("/", expand=False)
# split_cat = pd.DataFrame(df['new_category'].tolist(), columns=['category1', 'category2'])
# #each project should at least have 1 category, 'category2' can be "None". 
# #'category1' being the main category for the project.
# df = pd.concat([df, split_cat], axis=1)
# df.head()